In [31]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [32]:
game = pd.read_csv('rawg_games_20k.csv')

In [33]:
game

,id,name,slug,released,metacritic,rating,ratings_count,platforms,genres,stores,added,tags
0,3498,Grand Theft Auto V,grand-theft-auto-v,2013-09-17,92.0,4.47,7236,PC|PlayStation 5|Xbox Series S/X|PlayStation 4...,Action,Steam|PlayStation Store|Epic Games|Xbox 360 St...,22282,Singleplayer|Steam Achievements|Multiplayer|Fu...
1,3328,The Witcher 3: Wild Hunt,the-witcher-3-wild-hunt,2015-05-18,92.0,4.64,7052,PlayStation 5|Xbox Series S/X|macOS|PlayStatio...,Action|RPG,GOG|PlayStation Store|Steam|Xbox Store|Nintend...,21954,Singleplayer|Full controller support|Atmospher...
2,4200,Portal 2,portal-2,2011-04-18,95.0,4.58,5992,PlayStation 3|PC|Xbox 360|Linux|macOS|Xbox One,Shooter|Puzzle,Xbox Store|Steam|PlayStation Store|Xbox 360 Store,20717,Singleplayer|Steam Achievements|Multiplayer|Fu...
3,4291,Counter-Strike: Global Offensive,counter-strike-global-offensive,2012-08-21,81.0,3.57,3603,PC|Linux|Xbox 360|PlayStation 3,Shooter,PlayStation Store|Steam|Xbox 360 Store,18318,Steam Achievements|Multiplayer|Full controller...
4,5286,Tomb Raider (2013),tomb-raider,2013-03-05,86.0,4.06,4048,PlayStation 3|Xbox 360|macOS|PC|Xbox One|PlayS...,Action,Xbox 360 Store|Steam|PlayStation Store|Google ...,17697,Singleplayer|Multiplayer|Full controller suppo...
...,...,...,...,...,...,...,...,...,...,...,...,...
19995,17151,Fables from the Den,fables-from-the-den,2016-10-26,NaN,0.00,2,macOS|PC|Linux,NaN,Steam,60,Design & Illustration
19996,17108,Mysterious Space,mysterious-space,2015-05-15,NaN,0.00,1,PC,Action|Indie,Steam,60,Singleplayer|Steam Achievements|Full controlle...
19997,16530,侠客风云传(Tale of Wuxia),tale-of-wuxia,2016-04-28,NaN,0.00,5,PC,Adventure|RPG|Strategy|Simulation|Casual,Steam,60,Singleplayer|Steam Cloud|steam-trading-cards|R...
19998,16221,Killbot,killbot,2016-07-10,NaN,0.00,1,PC,Action|Adventure|Simulation|Massively Multipla...,Steam,60,Singleplayer|Multiplayer|steam-trading-cards|C...


In [34]:
game.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             20000 non-null  int64  
 1   name           20000 non-null  object 
 2   slug           20000 non-null  object 
 3   released       19328 non-null  object 
 4   metacritic     5406 non-null   float64
 5   rating         20000 non-null  float64
 6   ratings_count  20000 non-null  int64  
 7   platforms      19998 non-null  object 
 8   genres         19652 non-null  object 
 9   stores         18842 non-null  object 
 10  added          20000 non-null  int64  
 11  tags           19323 non-null  object 
dtypes: float64(2), int64(3), object(7)
memory usage: 1.8+ MB


In [35]:
game.describe()

,id,metacritic,rating,ratings_count,added
count,2.000000e+04,5406.000000,20000.000000,20000.000000,20000.000000
mean,1.378563e+05,73.747688,2.291999,84.126450,620.540450
std,2.402878e+05,11.087490,1.631992,278.981489,1325.524443
min,1.000000e+00,21.000000,0.000000,0.000000,60.000000
25%,1.415675e+04,68.000000,0.000000,4.000000,109.000000
50%,2.269200e+04,75.000000,2.880000,14.000000,193.500000
75%,6.138325e+04,81.000000,3.670000,49.000000,520.000000
max,1.007483e+06,99.000000,4.860000,7236.000000,22282.000000


In [36]:
game['name'].duplicated()

,name
0,False
1,False
2,False
3,False
4,False
...,...
19995,False
19996,False
19997,False
19998,False


In [37]:
game = game[['id', 'name', 'platforms', 'genres', 'stores', 'tags']]

In [38]:
game

,id,name,platforms,genres,stores,tags
0,3498,Grand Theft Auto V,PC|PlayStation 5|Xbox Series S/X|PlayStation 4...,Action,Steam|PlayStation Store|Epic Games|Xbox 360 St...,Singleplayer|Steam Achievements|Multiplayer|Fu...
1,3328,The Witcher 3: Wild Hunt,PlayStation 5|Xbox Series S/X|macOS|PlayStatio...,Action|RPG,GOG|PlayStation Store|Steam|Xbox Store|Nintend...,Singleplayer|Full controller support|Atmospher...
2,4200,Portal 2,PlayStation 3|PC|Xbox 360|Linux|macOS|Xbox One,Shooter|Puzzle,Xbox Store|Steam|PlayStation Store|Xbox 360 Store,Singleplayer|Steam Achievements|Multiplayer|Fu...
3,4291,Counter-Strike: Global Offensive,PC|Linux|Xbox 360|PlayStation 3,Shooter,PlayStation Store|Steam|Xbox 360 Store,Steam Achievements|Multiplayer|Full controller...
4,5286,Tomb Raider (2013),PlayStation 3|Xbox 360|macOS|PC|Xbox One|PlayS...,Action,Xbox 360 Store|Steam|PlayStation Store|Google ...,Singleplayer|Multiplayer|Full controller suppo...
...,...,...,...,...,...,...
19995,17151,Fables from the Den,macOS|PC|Linux,NaN,Steam,Design & Illustration
19996,17108,Mysterious Space,PC,Action|Indie,Steam,Singleplayer|Steam Achievements|Full controlle...
19997,16530,侠客风云传(Tale of Wuxia),PC,Adventure|RPG|Strategy|Simulation|Casual,Steam,Singleplayer|Steam Cloud|steam-trading-cards|R...
19998,16221,Killbot,PC,Action|Adventure|Simulation|Massively Multipla...,Steam,Singleplayer|Multiplayer|steam-trading-cards|C...


In [39]:
game['platforms'].isnull().sum()

np.int64(2)

In [40]:
game['genres'].isnull().sum()

np.int64(348)

In [41]:
game['id'].isnull().sum()

np.int64(0)

In [42]:
game['name'].isnull().sum()

np.int64(0)

In [43]:
game

,id,name,platforms,genres,stores,tags
0,3498,Grand Theft Auto V,PC|PlayStation 5|Xbox Series S/X|PlayStation 4...,Action,Steam|PlayStation Store|Epic Games|Xbox 360 St...,Singleplayer|Steam Achievements|Multiplayer|Fu...
1,3328,The Witcher 3: Wild Hunt,PlayStation 5|Xbox Series S/X|macOS|PlayStatio...,Action|RPG,GOG|PlayStation Store|Steam|Xbox Store|Nintend...,Singleplayer|Full controller support|Atmospher...
2,4200,Portal 2,PlayStation 3|PC|Xbox 360|Linux|macOS|Xbox One,Shooter|Puzzle,Xbox Store|Steam|PlayStation Store|Xbox 360 Store,Singleplayer|Steam Achievements|Multiplayer|Fu...
3,4291,Counter-Strike: Global Offensive,PC|Linux|Xbox 360|PlayStation 3,Shooter,PlayStation Store|Steam|Xbox 360 Store,Steam Achievements|Multiplayer|Full controller...
4,5286,Tomb Raider (2013),PlayStation 3|Xbox 360|macOS|PC|Xbox One|PlayS...,Action,Xbox 360 Store|Steam|PlayStation Store|Google ...,Singleplayer|Multiplayer|Full controller suppo...
...,...,...,...,...,...,...
19995,17151,Fables from the Den,macOS|PC|Linux,NaN,Steam,Design & Illustration
19996,17108,Mysterious Space,PC,Action|Indie,Steam,Singleplayer|Steam Achievements|Full controlle...
19997,16530,侠客风云传(Tale of Wuxia),PC,Adventure|RPG|Strategy|Simulation|Casual,Steam,Singleplayer|Steam Cloud|steam-trading-cards|R...
19998,16221,Killbot,PC,Action|Adventure|Simulation|Massively Multipla...,Steam,Singleplayer|Multiplayer|steam-trading-cards|C...


In [44]:
game['stores'].isnull().sum()

np.int64(1158)

In [45]:
game['tags'].isnull().sum()

np.int64(677)

In [47]:
game = game.dropna(subset=['genres', 'stores', 'tags', 'platforms'])

In [48]:
game

,id,name,platforms,genres,stores,tags
0,3498,Grand Theft Auto V,PC|PlayStation 5|Xbox Series S/X|PlayStation 4...,Action,Steam|PlayStation Store|Epic Games|Xbox 360 St...,Singleplayer|Steam Achievements|Multiplayer|Fu...
1,3328,The Witcher 3: Wild Hunt,PlayStation 5|Xbox Series S/X|macOS|PlayStatio...,Action|RPG,GOG|PlayStation Store|Steam|Xbox Store|Nintend...,Singleplayer|Full controller support|Atmospher...
2,4200,Portal 2,PlayStation 3|PC|Xbox 360|Linux|macOS|Xbox One,Shooter|Puzzle,Xbox Store|Steam|PlayStation Store|Xbox 360 Store,Singleplayer|Steam Achievements|Multiplayer|Fu...
3,4291,Counter-Strike: Global Offensive,PC|Linux|Xbox 360|PlayStation 3,Shooter,PlayStation Store|Steam|Xbox 360 Store,Steam Achievements|Multiplayer|Full controller...
4,5286,Tomb Raider (2013),PlayStation 3|Xbox 360|macOS|PC|Xbox One|PlayS...,Action,Xbox 360 Store|Steam|PlayStation Store|Google ...,Singleplayer|Multiplayer|Full controller suppo...
...,...,...,...,...,...,...
19994,18168,Lepofrenia,PC,Adventure|RPG|Indie,Steam,Singleplayer|Steam Achievements|steam-trading-...
19996,17108,Mysterious Space,PC,Action|Indie,Steam,Singleplayer|Steam Achievements|Full controlle...
19997,16530,侠客风云传(Tale of Wuxia),PC,Adventure|RPG|Strategy|Simulation|Casual,Steam,Singleplayer|Steam Cloud|steam-trading-cards|R...
19998,16221,Killbot,PC,Action|Adventure|Simulation|Massively Multipla...,Steam,Singleplayer|Multiplayer|steam-trading-cards|C...


In [56]:
def blank_space_remover(obj):
  return obj.str.replace(' ', '', regex=False)

In [57]:
game[['platforms', 'genres', 'stores', 'tags']] = game[['platforms', 'genres', 'stores', 'tags']].apply(blank_space_remover)

/tmp/ipython-input-1393245905.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game[['platforms', 'genres', 'stores', 'tags']] = game[['platforms', 'genres', 'stores', 'tags']].apply(blank_space_remover)


In [58]:
def pipe_remover(obj):
  return obj.str.replace('|', ' ', regex=False)

In [59]:
game[['platforms', 'genres', 'stores', 'tags']] = game[['platforms', 'genres', 'stores', 'tags']].apply(pipe_remover)

/tmp/ipython-input-1467880456.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game[['platforms', 'genres', 'stores', 'tags']] = game[['platforms', 'genres', 'stores', 'tags']].apply(pipe_remover)


In [60]:
game

,id,name,platforms,genres,stores,tags
0,3498,Grand Theft Auto V,PC PlayStation5 XboxSeriesS/X PlayStation4 Pla...,Action,Steam PlayStationStore EpicGames Xbox360Store ...,PC PlayStation5 XboxSeriesS/X PlayStation4 Pla...
1,3328,The Witcher 3: Wild Hunt,PlayStation5 XboxSeriesS/X macOS PlayStation4 ...,Action RPG,GOG PlayStationStore Steam XboxStore NintendoS...,PlayStation5 XboxSeriesS/X macOS PlayStation4 ...
2,4200,Portal 2,PlayStation3 PC Xbox360 Linux macOS XboxOne,Shooter Puzzle,XboxStore Steam PlayStationStore Xbox360Store,PlayStation3 PC Xbox360 Linux macOS XboxOneSho...
3,4291,Counter-Strike: Global Offensive,PC Linux Xbox360 PlayStation3,Shooter,PlayStationStore Steam Xbox360Store,PC Linux Xbox360 PlayStation3ShooterPlayStatio...
4,5286,Tomb Raider (2013),PlayStation3 Xbox360 macOS PC XboxOne PlayStat...,Action,Xbox360Store Steam PlayStationStore GooglePlay...,PlayStation3 Xbox360 macOS PC XboxOne PlayStat...
...,...,...,...,...,...,...
19994,18168,Lepofrenia,PC,Adventure RPG Indie,Steam,PCAdventure RPG IndieSteamSingleplayer SteamAc...
19996,17108,Mysterious Space,PC,Action Indie,Steam,PCAction IndieSteamSingleplayer SteamAchieveme...
19997,16530,侠客风云传(Tale of Wuxia),PC,Adventure RPG Strategy Simulation Casual,Steam,PCAdventure RPG Strategy Simulation CasualStea...
19998,16221,Killbot,PC,Action Adventure Simulation MassivelyMultiplay...,Steam,PCAction Adventure Simulation MassivelyMultipl...


In [61]:
game['tags'] = game['platforms'] + ' ' + game['genres'] + ' ' + game['stores'] + ' ' + game['tags']

/tmp/ipython-input-937292035.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game['tags'] = game['platforms'] + ' ' + game['genres'] + ' ' + game['stores'] + ' ' + game['tags']


In [62]:
game

,id,name,platforms,genres,stores,tags
0,3498,Grand Theft Auto V,PC PlayStation5 XboxSeriesS/X PlayStation4 Pla...,Action,Steam PlayStationStore EpicGames Xbox360Store ...,PC PlayStation5 XboxSeriesS/X PlayStation4 Pla...
1,3328,The Witcher 3: Wild Hunt,PlayStation5 XboxSeriesS/X macOS PlayStation4 ...,Action RPG,GOG PlayStationStore Steam XboxStore NintendoS...,PlayStation5 XboxSeriesS/X macOS PlayStation4 ...
2,4200,Portal 2,PlayStation3 PC Xbox360 Linux macOS XboxOne,Shooter Puzzle,XboxStore Steam PlayStationStore Xbox360Store,PlayStation3 PC Xbox360 Linux macOS XboxOne Sh...
3,4291,Counter-Strike: Global Offensive,PC Linux Xbox360 PlayStation3,Shooter,PlayStationStore Steam Xbox360Store,PC Linux Xbox360 PlayStation3 Shooter PlayStat...
4,5286,Tomb Raider (2013),PlayStation3 Xbox360 macOS PC XboxOne PlayStat...,Action,Xbox360Store Steam PlayStationStore GooglePlay...,PlayStation3 Xbox360 macOS PC XboxOne PlayStat...
...,...,...,...,...,...,...
19994,18168,Lepofrenia,PC,Adventure RPG Indie,Steam,PC Adventure RPG Indie Steam PCAdventure RPG I...
19996,17108,Mysterious Space,PC,Action Indie,Steam,PC Action Indie Steam PCAction IndieSteamSingl...
19997,16530,侠客风云传(Tale of Wuxia),PC,Adventure RPG Strategy Simulation Casual,Steam,PC Adventure RPG Strategy Simulation Casual St...
19998,16221,Killbot,PC,Action Adventure Simulation MassivelyMultiplay...,Steam,PC Action Adventure Simulation MassivelyMultip...


In [63]:
game = game[['id', 'name', 'tags']]

In [64]:
game

,id,name,tags
0,3498,Grand Theft Auto V,PC PlayStation5 XboxSeriesS/X PlayStation4 Pla...
1,3328,The Witcher 3: Wild Hunt,PlayStation5 XboxSeriesS/X macOS PlayStation4 ...
2,4200,Portal 2,PlayStation3 PC Xbox360 Linux macOS XboxOne Sh...
3,4291,Counter-Strike: Global Offensive,PC Linux Xbox360 PlayStation3 Shooter PlayStat...
4,5286,Tomb Raider (2013),PlayStation3 Xbox360 macOS PC XboxOne PlayStat...
...,...,...,...
19994,18168,Lepofrenia,PC Adventure RPG Indie Steam PCAdventure RPG I...
19996,17108,Mysterious Space,PC Action Indie Steam PCAction IndieSteamSingl...
19997,16530,侠客风云传(Tale of Wuxia),PC Adventure RPG Strategy Simulation Casual St...
19998,16221,Killbot,PC Action Adventure Simulation MassivelyMultip...


In [65]:
game['tags'][0]

'PC PlayStation5 XboxSeriesS/X PlayStation4 PlayStation3 Xbox360 XboxOne Action Steam PlayStationStore EpicGames Xbox360Store XboxStore PC PlayStation5 XboxSeriesS/X PlayStation4 PlayStation3 Xbox360 XboxOneActionSteam PlayStationStore EpicGames Xbox360Store XboxStoreSingleplayer SteamAchievements Multiplayer Fullcontrollersupport Atmospheric GreatSoundtrack RPG Co-op OpenWorld cooperative First-Person ThirdPerson Funny Sandbox Comedy Third-PersonShooter Moddable Crime vrmod'

In [66]:
game['tags'] = game['tags'].str.lower()

/tmp/ipython-input-765118275.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game['tags'] = game['tags'].str.lower()


In [67]:
game

,id,name,tags
0,3498,Grand Theft Auto V,pc playstation5 xboxseriess/x playstation4 pla...
1,3328,The Witcher 3: Wild Hunt,playstation5 xboxseriess/x macos playstation4 ...
2,4200,Portal 2,playstation3 pc xbox360 linux macos xboxone sh...
3,4291,Counter-Strike: Global Offensive,pc linux xbox360 playstation3 shooter playstat...
4,5286,Tomb Raider (2013),playstation3 xbox360 macos pc xboxone playstat...
...,...,...,...
19994,18168,Lepofrenia,pc adventure rpg indie steam pcadventure rpg i...
19996,17108,Mysterious Space,pc action indie steam pcaction indiesteamsingl...
19997,16530,侠客风云传(Tale of Wuxia),pc adventure rpg strategy simulation casual st...
19998,16221,Killbot,pc action adventure simulation massivelymultip...


In [76]:
cv = CountVectorizer(max_features=1000, stop_words='english')
vectors = cv.fit_transform(game['tags']).toarray()

In [75]:
vectors.shape

(18285, 4416)

In [77]:
game = game.reset_index(drop=True)

In [78]:
similarity = cosine_similarity(vectors)

In [79]:
similarity

array([[1.        , 0.5490302 , 0.51456122, ..., 0.15188706, 0.19805348,
        0.28207597],
       [0.5490302 , 1.        , 0.34604988, ..., 0.28941438, 0.14675988,
        0.47546648],
       [0.51456122, 0.34604988, 1.        , ..., 0.23249528, 0.37333931,
        0.42624134],
       ...,
       [0.15188706, 0.28941438, 0.23249528, ..., 1.        , 0.36623351,
        0.52777778],
       [0.19805348, 0.14675988, 0.37333931, ..., 0.36623351, 1.        ,
        0.30988989],
       [0.28207597, 0.47546648, 0.42624134, ..., 0.52777778, 0.30988989,
        1.        ]])

In [80]:
def recommend(games):
  game_index = game[game['name'] == games].index[0]
  distances = similarity[game_index]
  games_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]
  for i in games_list:
        print(game.iloc[i[0]]['name'])

In [81]:
recommend('Mysterious Space')

Chompy Chomp Chomp
Operation KREEP
Jamsouls
IACTURA
Horizon Shift
